<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-08 13:09:29
-------------------
qualified stocks: 94
with latest results: 87
still star stocks: 47


-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  6.31 L
Current:  1.24 C
-------------------
Today PnL: 27.91 K (0.23%)
Current PnL: -21.89 L (-15.52%)
CY Booked + Current PnL: -19.05 L (-13.51%)
-------------------
Total profit:  2.99 L
Total loss:  -24.88 L
-------------------
Total Booked + Current PnL: 7.93 L (6.95%)
Total Booked PnL: 29.81 L (26.14%)
Curr Year Booked PnL: 2.83 L (2.29%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.05 C
Est FTT PnL: 81.08 L (65.59%)
-------------------
Est LTT:  2.66 C
Est LTT PnL: 1.42 C (115.03%)
Deployed:  1.14 C
Current:  1.24 C
CAGR/XIRR %: 5.87%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MRF,114777.60,150504.14,21.99,X-MC,97.87,140080.0,25302.0,10422.0,-0.24,22.04,7.44,31.13,203.0,2.43,1.19,36.45,AR,ATH,TYRES
70,SBILIFE,1496.49,1924.99,-1.84,H-LC,87.77,145883.0,20178.0,15814.0,-0.45,16.05,10.84,28.63,116.0,1.28,1.24,29.96,AR,ATH,INSURANCE
57,PIDILITIND,2504.06,3576.00,-14.91,X-LC,29.79,88830.0,13708.0,18450.0,-0.48,18.25,20.77,42.81,66.0,0.74,0.76,11.40,X40,BTT,CHEMICALS
71,SBIN,760.30,863.00,-17.60,M-LC,36.17,206832.0,5352.0,21862.0,0.57,2.66,10.57,13.51,106.0,0.24,1.76,13.31,XY25,NTT,BANKS
35,ICICIGI,1852.76,2268.71,-19.16,H-LC,35.11,97276.0,-2773.0,25233.0,-1.35,-2.77,25.94,22.45,93.0,-0.11,0.83,18.03,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,CERA,9059.90,10420.45,80.42,X-SC,37.23,55595.0,-35004.0,48612.0,2.93,-38.64,87.44,15.02,50.0,-0.72,0.47,9.39,X40N,ATH,CERAMICS
88,VAIBHAVGBL,350.21,521.00,42.15,X-SC,68.09,94699.0,-57292.0,131414.0,2.66,-37.69,138.77,48.77,18.0,-0.44,0.81,8.77,XR,NTT,JEWELLERY
49,LUXIND,1448.19,2442.00,27.37,M-SC,45.74,131926.0,-5652.0,100066.0,2.52,-4.11,75.85,68.62,30.0,-0.06,1.12,14.37,X40N,NTT,TEXTILES
9,ATULAUTO,624.44,844.00,3429.73,M-SC,7.45,110059.0,-37309.0,89126.0,2.52,-25.32,80.98,35.16,188.0,-0.42,0.94,12.41,XY24,NTT,AUTO
90,VIPIND,488.80,718.00,-777.97,H-SC,98.94,64565.0,-30751.0,75444.0,2.35,-32.26,116.85,46.89,39.0,-0.41,0.55,27.35,X40N,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,SIEMENS,4406.90,7969.85,-20.65,X-LC,46.81,71725.0,-38447.0,127520.0,-3.31,-34.90,177.79,80.85,20.0,-0.30,0.61,5.64,AR,ATH,ELECTRICAL
22,DABUR,505.20,735.00,-16.89,X-MC,40.43,182910.0,-14118.0,103747.0,-2.72,-7.17,56.72,45.49,81.0,-0.14,1.56,3.90,XY24,BTT,FMCG
21,COLPAL,2837.00,3808.06,4.94,X-MC,55.32,128310.0,-13540.0,62089.0,-2.60,-9.55,48.39,34.23,36.0,-0.22,1.09,9.18,X40,ATH,FMCG
20,COFFEEDAY,59.14,80.00,-61.77,L-SC,72.34,57830.0,-55719.0,95766.0,-2.02,-49.07,165.60,35.27,151.0,-0.58,0.49,41.54,XR,NTT,HOTELS
54,NHPC,82.08,115.00,189.95,M-MC,17.02,140773.0,-240.0,56802.0,-1.76,-0.17,40.35,40.11,211.0,-0.00,1.20,15.08,XY24,NTT,POWER


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TATAMOTORS,798.82,1065.00,-54.49,M-LC,87.77,140866.0,-22892.0,77462.0,1.01,-13.98,54.99,33.32,1.0,-0.30,1.20,18.53,XY24,NTT,AUTO
79,TANLA,991.56,1963.11,-57.60,H-SC,71.28,107815.0,-111320.0,326033.0,2.22,-50.80,302.40,97.98,2.0,-0.34,0.92,16.78,AR,ATH,IT
2,ABB,5319.37,8953.12,-40.75,H-LC,67.02,134562.0,1578.0,89268.0,-0.28,1.19,66.34,68.31,3.0,0.02,1.15,9.72,AR,ATH,ELECTRICAL
24,DLF,684.82,1031.70,-26.56,H-LC,74.47,101385.0,-1338.0,53369.0,-0.49,-1.30,52.64,50.65,4.0,-0.03,0.86,11.16,AR,ATH,REALTY
10,AWL,361.39,485.00,-69.22,H-MC,8.51,135600.0,-50516.0,114175.0,0.63,-27.14,84.20,34.20,6.0,-0.44,1.16,11.69,XY24,NTT,FMCG


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MRF,114777.60,150504.14,21.99,X-MC,97.87,140080.0,25302.0,10422.0,-0.24,22.04,7.44,31.13,203.0,2.43,1.19,36.45,AR,ATH,TYRES
4,ADANIPORTS,1103.69,1583.00,4.69,M-LC,92.02,184758.0,33552.0,32111.0,0.78,22.19,17.38,43.43,45.0,1.04,1.57,28.07,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,POLYCAB,5005.92,7541.35,15.83,H-MC,90.43,164360.0,24194.0,46793.0,-0.28,17.26,28.47,50.65,74.0,0.52,1.40,25.44,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,4.69,M-LC,92.02,184758.0,33552.0,32111.0,0.78,22.19,17.38,43.43,45.0,1.04,1.57,28.07,XY24,NTT,MISC
6,ANGELONE,2328.67,3033.00,-4.34,H-SC,70.21,155942.0,4578.0,41200.0,0.99,3.02,26.42,30.25,108.0,0.11,1.33,23.42,X40N,NTT,FINANCE
56,PFIZER,4217.93,6318.30,-14.57,H-SC,63.83,103130.0,1900.0,48512.0,0.50,1.88,47.04,49.80,89.0,0.04,0.88,9.79,X40,ATH,PHARMA
71,SBIN,760.30,863.00,-17.60,M-LC,36.17,206832.0,5352.0,21862.0,0.57,2.66,10.57,13.51,106.0,0.24,1.76,13.31,XY25,NTT,BANKS


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,-7.88,H-SC,38.30,101951.0,-53.0,45073.0,-0.04,-0.05,44.21,44.14,26.0,-0.00,0.87,19.39,XR,ATH,MISC
93,WIPRO,243.46,420.00,-12.43,M-LC,13.83,149972.0,-973.0,110424.0,-0.88,-0.64,73.63,72.51,77.0,-0.01,1.28,12.92,XR,NTT,IT
12,BANKINDIA,113.49,190.00,-28.40,M-MC,31.91,125795.0,-3357.0,90421.0,1.24,-2.60,71.88,67.42,99.0,-0.04,1.07,22.35,XR,NTT,BANKS
91,VOLTAS,1272.00,1929.20,-19.83,H-MC,6.38,136433.0,-3487.0,75775.0,-0.30,-2.49,55.54,51.67,9.0,-0.05,1.16,3.05,AR,ATH,AC
1,AARTIIND,487.04,919.00,20.82,M-SC,100.00,76444.0,-4892.0,77025.0,-0.15,-6.01,100.76,88.69,114.0,-0.06,0.65,28.91,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,UJJIVANSFB,52.77,60.00,-42.16,M-SC,79.79,111996.0,-30483.0,50006.0,1.74,-21.39,44.65,13.70,139.0,-0.61,0.95,32.19,X40N,NTT,BANKS
52,MRF,114777.60,150504.14,21.99,X-MC,97.87,140080.0,25302.0,10422.0,-0.24,22.04,7.44,31.13,203.0,2.43,1.19,36.45,AR,ATH,TYRES
70,SBILIFE,1496.49,1924.99,-1.84,H-LC,87.77,145883.0,20178.0,15814.0,-0.45,16.05,10.84,28.63,116.0,1.28,1.24,29.96,AR,ATH,INSURANCE
51,MEDANTA,1022.60,1486.00,4.65,X-MC,14.89,184406.0,28971.0,41473.0,0.19,18.64,22.49,45.32,189.0,0.70,1.57,25.47,XY24,NTT,HEALTHCARE
61,RBLBANK,230.61,388.00,33.53,M-SC,94.68,207416.0,-26884.0,186799.0,1.35,-11.47,90.06,68.25,207.0,-0.14,1.77,36.59,XY24,NTT,BANKS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,AARTIIND,487.04,919.00,20.82,M-SC,100.00,76444.0,-4892.0,77025.0,-0.15,-6.01,100.76,88.69,114.0,-0.06,0.65,28.91,XR,NTT,CHEMICALS
90,VIPIND,488.80,718.00,-777.97,H-SC,98.94,64565.0,-30751.0,75444.0,2.35,-32.26,116.85,46.89,39.0,-0.41,0.55,27.35,X40N,NTT,MISC
52,MRF,114777.60,150504.14,21.99,X-MC,97.87,140080.0,25302.0,10422.0,-0.24,22.04,7.44,31.13,203.0,2.43,1.19,36.45,AR,ATH,TYRES
76,SONACOMS,578.05,1006.00,-31.35,M-MC,96.81,89250.0,-11909.0,86796.0,-0.22,-11.77,97.25,74.03,7.0,-0.14,0.76,25.43,AR,BTT,AUTO
38,INDUSINDBK,1354.50,1800.00,-36.29,H-MC,95.74,47051.0,-30156.0,55548.0,-1.00,-39.06,118.06,32.89,46.0,-0.54,0.40,29.57,XR,NTT,BANKS


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.86
1,25,41.77
2,50,69.71


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    35.93
X40N    14.04
X40     13.62
AR      12.09
XY25    11.43
XR      10.48
X200     1.18
SR       1.17
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.42
X-LC    18.80
M-SC    16.46
M-LC     8.10
M-MC     7.65
X-MC     6.99
H-MC     6.41
H-LC     5.59
X-SC     2.63
L-SC     2.27
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
BANKS,11.30,-13.46,69.10
FMCG,11.13,-3.27,34.89
IT,9.09,-27.63,94.44
FINANCE,7.52,-25.58,82.25
MISC,6.51,-34.83,83.61
PAINTS,5.80,-20.93,56.72
INSURANCE,5.53,-7.84,40.38
ELECTRICAL,4.19,-13.21,70.90
REFINERIES,3.82,9.05,7.72


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3210164.0,28
XR,1315437.0,15
X40N,1250388.0,17
AR,1075835.0,13
X40,774935.0,12
SR,207354.0,2
XY25,195352.0,6
X200,79049.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2337592.0,21
M-SC,1990167.0,20
X-LC,776375.0,12
M-MC,692672.0,7
H-MC,369769.0,6
X-SC,369019.0,4
M-LC,364833.0,6
X-MC,333050.0,6
L-SC,331303.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      964765.0      6
M-SC       XY24      808067.0      6
H-SC       X40N      582681.0      7
M-MC       XY24      489243.0      4
M-SC       XR        448340.0      5
X-LC       X40       400157.0      5
M-SC       X40N      390681.0      5
H-SC       AR        388998.0      2
           XR        311881.0      4
M-LC       XY24      232547.0      4
X-LC       AR        192996.0      3
L-SC       XR        167409.0      2
M-SC       X40       164564.0      2
H-LC       AR        158451.0      3
H-MC       XY24      158028.0      2
X-MC       XY24      145220.0      2
X-SC       XR        131414.0      1
           SR        122691.0      1
X-MC       X40N      115319.0      2
X-SC       X40N      114914.0      2
M-LC       XR        110424.0      1
X-LC       XY25      104173.0      3
L-SC       XY24       95349.0      1
M-SC       AR         93852.0      1
M-MC       XR         90421.0      1
H-SC       X40        89267.0      2
M-MC       AR         86796.0      1
M-SC       SR         84663.0      1
X-LC       X200       79049.0      1
H-MC       AR         75775.0      1
L-SC       AR         68545.0      1
X-MC       X40        62089.0      1
H-MC       XR         55548.0      1
           X40N       46793.0      1
H-LC       XY25       43105.0      1
H-MC       X40        33625.0      1
M-MC       XY25       26212.0      1
H-LC       X40        25233.0      1
M-LC       XY25       21862.0      1
X-MC       AR         10422.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
